In [1]:
import sys, oqs
print("Python:", sys.executable)
print("oqs module:", oqs.__file__)

kems = oqs.get_enabled_kem_mechanisms()
sigs = oqs.get_enabled_sig_mechanisms()
print("HQC entries:", [k for k in kems if "HQC" in k])
print("Dil entries:", [k for k in sigs if "Dilithium" in k])
# print("Count:", len(sigs))
print(sigs)
# print(sys.executable)
# !{sys.executable} -m pip -V

#!{sys.executable} -m pip install scikit-learn

import sklearn
#print(sklearn.__version__)


/Users/newspeed/.venvs/ds/lib/python3.12/site-packages/oqs/__init__.py:1: UserWarning: liboqs version (major, minor) 0.15.0 differs from liboqs-python version 0.14.0
  from oqs.oqs import (


Python: /Users/newspeed/.venvs/ds/bin/python3
oqs module: /Users/newspeed/.venvs/ds/lib/python3.12/site-packages/oqs/__init__.py
HQC entries: []
Dil entries: []
('ML-DSA-44', 'ML-DSA-65', 'ML-DSA-87', 'Falcon-512', 'Falcon-1024', 'Falcon-padded-512', 'Falcon-padded-1024', 'SPHINCS+-SHA2-128f-simple', 'SPHINCS+-SHA2-128s-simple', 'SPHINCS+-SHA2-192f-simple', 'SPHINCS+-SHA2-192s-simple', 'SPHINCS+-SHA2-256f-simple', 'SPHINCS+-SHA2-256s-simple', 'SPHINCS+-SHAKE-128f-simple', 'SPHINCS+-SHAKE-128s-simple', 'SPHINCS+-SHAKE-192f-simple', 'SPHINCS+-SHAKE-192s-simple', 'SPHINCS+-SHAKE-256f-simple', 'SPHINCS+-SHAKE-256s-simple', 'MAYO-1', 'MAYO-2', 'MAYO-3', 'MAYO-5', 'cross-rsdp-128-balanced', 'cross-rsdp-128-fast', 'cross-rsdp-128-small', 'cross-rsdp-192-balanced', 'cross-rsdp-192-fast', 'cross-rsdp-192-small', 'cross-rsdp-256-balanced', 'cross-rsdp-256-fast', 'cross-rsdp-256-small', 'cross-rsdpg-128-balanced', 'cross-rsdpg-128-fast', 'cross-rsdpg-128-small', 'cross-rsdpg-192-balanced', 'cross

In [2]:
# QUICK DATASET SMOKE TEST

import os, glob, sys
import pandas as pd

# 1) Find data files (looks under ./data first, then anywhere under cwd)
candidates = []
for pattern in [
    "data/**/*.csv",
    "**/*.csv",
]:
    candidates += glob.glob(pattern, recursive=True)

# De-dupe and keep readable paths
candidates = sorted(set(candidates))
print(f"Found {len(candidates)} data file(s).")
for i, p in enumerate(candidates[:10], 1):
    print(f"{i:>2}. {p}")

assert candidates, "No CSV files found. Did you unzip the dataset?"

# Get the paths of the Kaggle CSVs
csvs   = [p for p in candidates if "benchmark" in p and p.lower().endswith(".csv")]
path   = (csvs)[0]
path2   = (csvs)[1]
print("\nLoading:", path)
print("\nLoading:", path2)

# Load Kaggle data
kaggle_df = pd.read_csv(path, nrows=50_000)   # cap rows for quick test
kaggle_df2 = pd.read_csv(path2, nrows=50_000) 

# Basic checks
print("\nShape:", kaggle_df.shape)
print("Columns:", list(kaggle_df.columns)[:20], "..." if len(kaggle_df.columns) > 20 else "")
print("\nNulls (top 10 cols):")
print(kaggle_df.isna().sum().sort_values(ascending=False).head(10))

display(kaggle_df.head())

# Light profiling
print("\nSample stats (numeric columns):")
display(kaggle_df.select_dtypes("number").describe().T.head(10))

print("\nValue counts preview (first non-numeric column):")
non_num = [c for c in kaggle_df.columns if kaggle_df[c].dtype == "object"]
if non_num:
    display(kaggle_df[non_num[0]].value_counts().head(10))
else:
    print("(no object/categorical columns found)")

print("\nShape:", kaggle_df.shape)
print("Columns:", list(kaggle_df.columns)[:20], "..." if len(kaggle_df.columns) > 20 else "")
print("\nNulls (top 10 cols):")
print(kaggle_df.isna().sum().sort_values(ascending=False).head(10))

display(kaggle_df2.head())

# Light profiling
print("\nSample stats (numeric columns):")
display(kaggle_df2.select_dtypes("number").describe().T.head(10))

print("\nValue counts preview (first non-numeric column):")
non_num = [c for c in kaggle_df2.columns if kaggle_df2[c].dtype == "object"]
if non_num:
    display(kaggle_df2[non_num[0]].value_counts().head(10))
else:
    print("(no object/categorical columns found)")

print("\nShape:", kaggle_df2.shape)
print("Columns:", list(kaggle_df2.columns)[:20], "..." if len(kaggle_df2.columns) > 20 else "")
print("\nNulls (top 10 cols):")
print(kaggle_df.isna().sum().sort_values(ascending=False).head(10))


Found 10 data file(s).
 1. data/pqc/kem_benchmark_results.csv
 2. data/pqc/pqc_perf_data.csv
 3. data/pqc/pqc_perf_data_corrupted.csv
 4. data/pqc/pqc_perf_data_kem_local.csv
 5. data/pqc/pqc_perf_data_sig_local.csv
 6. data/pqc/pqc_perf_data_test.csv
 7. data/pqc/pqc_perf_schema.csv
 8. data/pqc/signature_benchmark_results.csv
 9. datapqcpqc_perf_data_kem_local.csv
10. datapqcpqc_perf_data_sig_local.csv

Loading: data/pqc/kem_benchmark_results.csv

Loading: data/pqc/signature_benchmark_results.csv

Shape: (37004, 16)
Columns: ['algorithm', 'keygen_time_ms', 'encap_time_ms', 'decap_time_ms', 'ciphertext_length', 'shared_secret_length', 'overhead_bytes', 'total_time_ms', 'correctness', 'type', 'text_size_kb', 'text_length_bytes', 'run_id', 'timestamp', 'security_level_bits', 'security_level'] 

Nulls (top 10 cols):
algorithm               0
keygen_time_ms          0
encap_time_ms           0
decap_time_ms           0
ciphertext_length       0
shared_secret_length    0
overhead_bytes    

,algorithm,keygen_time_ms,encap_time_ms,decap_time_ms,ciphertext_length,shared_secret_length,overhead_bytes,total_time_ms,correctness,type,text_size_kb,text_length_bytes,run_id,timestamp,security_level_bits,security_level
0,BIKE-L1,8.220911,0.476837,6.126165,1573,32,1541,14.823914,True,KEM,0.5,512,1,2025-04-07T09:04:19.790636,128,1
1,BIKE-L3,21.829844,1.256943,16.112328,3115,32,3083,39.199114,True,KEM,0.5,512,1,2025-04-07T09:04:19.829883,192,2
2,BIKE-L5,46.767712,2.524376,37.101746,5154,32,5122,86.393833,True,KEM,0.5,512,1,2025-04-07T09:04:19.916316,256,3
3,Classic-McEliece-348864,93.240976,0.107050,17.526865,96,32,64,110.874891,True,KEM,0.5,512,1,2025-04-07T09:04:20.027250,128,1
4,Classic-McEliece-348864f,29.181004,0.037670,16.361237,96,32,64,45.579910,True,KEM,0.5,512,1,2025-04-07T09:04:20.072888,128,1



Sample stats (numeric columns):


,count,mean,std,min,25%,50%,75%,max
keygen_time_ms,37004.0,71.456538,1283.209902,0.009775,0.416994,2.707243,85.134983,245504.775763
encap_time_ms,37004.0,0.924705,1.429549,0.009775,0.041246,0.184774,1.038074,9.560108
decap_time_ms,37004.0,20.280116,26.511202,0.006676,0.315189,4.781008,33.585608,244.152069
ciphertext_length,37004.0,4878.517241,6662.425115,96.000000,208.000000,1088.000000,8978.000000,21632.000000
shared_secret_length,37004.0,33.655172,11.192131,16.000000,32.000000,32.000000,32.000000,64.000000
overhead_bytes,37004.0,4844.862069,6661.950499,64.000000,176.000000,1056.000000,8914.000000,21600.000000
total_time_ms,37004.0,92.661359,1286.541919,0.027895,1.753092,11.262178,118.201017,245749.175072
text_size_kb,37004.0,64.250000,36.835434,0.500000,32.375000,64.250000,96.125000,128.000000
text_length_bytes,37004.0,65791.600313,37719.483995,512.000000,33151.500000,65791.500000,98431.500000,131072.000000
run_id,37004.0,638.500000,368.354336,1.000000,319.750000,638.500000,957.250000,1276.000000



Value counts preview (first non-numeric column):


...


Sample stats (numeric columns):


,count,mean,std,min,25%,50%,75%,max
text_size_kb,50000.0,45.092864,25.774819,0.500000,22.800000,45.100000,67.400000,89.700000
keygen_time_ms,50000.0,8.820116,21.455216,0.006676,0.030041,0.497818,3.674746,143.803835
sign_time_ms,50000.0,72.870953,204.097767,0.021696,0.252724,0.864029,4.601121,1243.484259
verify_time_ms,50000.0,0.674328,0.605104,0.023127,0.186920,0.473976,1.002073,3.992796
signature_length,50000.0,15874.615100,18316.320976,96.000000,650.000000,8960.000000,29792.000000,74590.000000
overhead_bytes,50000.0,-30300.077700,32122.737261,-91756.000000,-55201.500000,-30542.000000,-7442.000000,74078.000000
total_time_ms,50000.0,82.365397,224.971493,0.102997,1.197100,3.004074,13.875067,1388.270855
text_length_bytes,50000.0,46174.692800,26393.414035,512.000000,23347.000000,46182.000000,69017.000000,91852.000000
run_id,50000.0,446.928640,257.748190,1.000000,224.000000,447.000000,670.000000,893.000000
security_level_bits,50000.0,173.721600,53.703846,128.000000,128.000000,128.000000,256.000000,256.000000



Value counts preview (first non-numeric column):


algorithm
Dilithium2            893
Dilithium3            893
Dilithium5            893
ML-DSA-44             893
ML-DSA-65             893
ML-DSA-87             893
Falcon-512            893
Falcon-1024           893
Falcon-padded-512     893
Falcon-padded-1024    893
Name: count, dtype: int64


Shape: (50000, 15)
Columns: ['algorithm', 'text_size_kb', 'keygen_time_ms', 'sign_time_ms', 'verify_time_ms', 'signature_length', 'overhead_bytes', 'total_time_ms', 'correctness', 'type', 'text_length_bytes', 'run_id', 'timestamp', 'security_level_bits', 'security_level'] 

Nulls (top 10 cols):
algorithm               0
keygen_time_ms          0
encap_time_ms           0
decap_time_ms           0
ciphertext_length       0
shared_secret_length    0
overhead_bytes          0
total_time_ms           0
correctness             0
type                    0
dtype: int64


# Project 3 — PQC Performance Modeling

_Full starter notebook with Preprocessing → Modeling → Visualization_

## 1) Preprocessing
This section loads CSVs, cleans data, and engineers features ready for ML.

In [3]:

# 1.1 Configuration
KEM_CSV_PATH = "data/pqc/pqc_perf_data_kem_local.csv"   # Generated data
SIG_CSV_PATH = "data/pqc/pqc_perf_data_sig_local.csv"   # Generated data
TARGET = "latency_ms"            # or "throughput_ops" if you prefer
RANDOM_SEED = 42
PRINT_ROWS = 5

import os, sys, platform, numpy as np, pandas as pd
np.set_printoptions(edgeitems=5, linewidth=120)
pd.set_option("display.max_columns", 100)

print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("CSV_PATH:", KEM_CSV_PATH)
print("CSV_PATH:", SIG_CSV_PATH)


Python: 3.12.11
Platform: macOS-15.4.1-x86_64-i386-64bit
CSV_PATH: data/pqc/pqc_perf_data_kem_local.csv
CSV_PATH: data/pqc/pqc_perf_data_sig_local.csv


In [4]:

try:
    df_local_kem = pd.read_csv(KEM_CSV_PATH)
    df_local_sig = pd.read_csv(SIG_CSV_PATH)
    
    print("Loaded rows:", len(df_local_kem))
    # print("Columns:", list(df.columns)[:20])
    display(df_local_kem.head(PRINT_ROWS))
    print("Shape: ", df_local_sig.shape)
    print("Number of columns:", len(df_local_kem.columns))
    print(list(df_local_kem.columns))

    print("Loaded rows:", len(df_local_sig))
    # print("Columns:", list(df.columns)[:20])
    display(df_local_sig.head(PRINT_ROWS))
    print("Shape: ", df_local_sig.shape)
    print("Number of columns:", len(df_local_sig.columns))
    print(list(df_local_sig.columns))

except FileNotFoundError:
    raise SystemExit(f"CSV not found at {CSV_PATH}. Generate it with pqc_benchmark_harness.py.")


Loaded rows: 2340


,algorithm,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,crypto_type,correctness,timestamp,notes,trial_id
0,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.7687,1300.83,KEM,True,2025-11-20T22:36:03.573924,NaN,1
1,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.3240,3086.47,KEM,True,2025-11-20T22:36:03.574763,NaN,2
2,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2500,3999.89,KEM,True,2025-11-20T22:36:03.575045,NaN,3
3,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2230,4484.97,KEM,True,2025-11-20T22:36:03.575294,NaN,4
4,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2271,4404.20,KEM,True,2025-11-20T22:36:03.575545,NaN,5


Shape:  (1800, 18)
Number of columns: 18
['algorithm', 'operation', 'message_bytes', 'concurrency', 'cpu_ghz', 'aesni', 'avx2', 'neon', 'machine_id', 'os_name', 'containerized', 'latency_ms', 'throughput_ops', 'crypto_type', 'correctness', 'timestamp', 'notes', 'trial_id']
Loaded rows: 1800


,algorithm,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,crypto_type,correctness,timestamp,notes,trial_id
0,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,1.2456,802.80,Signature,True,2025-11-20T22:32:40.486951,NaN,1
1,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.1651,6058.40,Signature,True,2025-11-20T22:32:40.487678,NaN,2
2,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.1644,6081.43,Signature,True,2025-11-20T22:32:40.487976,NaN,3
3,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.1524,6559.70,Signature,True,2025-11-20T22:32:40.488194,NaN,4
4,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.1277,7833.12,Signature,True,2025-11-20T22:32:40.488405,NaN,5


Shape:  (1800, 18)
Number of columns: 18
['algorithm', 'operation', 'message_bytes', 'concurrency', 'cpu_ghz', 'aesni', 'avx2', 'neon', 'machine_id', 'os_name', 'containerized', 'latency_ms', 'throughput_ops', 'crypto_type', 'correctness', 'timestamp', 'notes', 'trial_id']


In [5]:
import pandas as pd

kem_raw = pd.read_csv("data/pqc/kem_benchmark_results.csv")
sig_raw = pd.read_csv("data/pqc/signature_benchmark_results.csv")

print("KEM shape:", kem_raw.shape)
print("Signature shape:", sig_raw.shape)
kem_raw.head(), sig_raw.head()

print("KEM algorithms from Kaggle:")
print(sorted(kem_raw["algorithm"].unique())[:30])   # first 30

print("Signature algorithms from Kaggle:")
print(sorted(sig_raw["algorithm"].unique())[:30])


KEM shape: (37004, 16)
Signature shape: (71456, 15)
KEM algorithms from Kaggle:
['BIKE-L1', 'BIKE-L3', 'BIKE-L5', 'Classic-McEliece-348864', 'Classic-McEliece-348864f', 'Classic-McEliece-460896', 'Classic-McEliece-460896f', 'Classic-McEliece-6688128', 'Classic-McEliece-6688128f', 'Classic-McEliece-6960119', 'Classic-McEliece-6960119f', 'Classic-McEliece-8192128', 'Classic-McEliece-8192128f', 'FrodoKEM-1344-AES', 'FrodoKEM-1344-SHAKE', 'FrodoKEM-640-AES', 'FrodoKEM-640-SHAKE', 'FrodoKEM-976-AES', 'FrodoKEM-976-SHAKE', 'HQC-128', 'HQC-192', 'HQC-256', 'Kyber1024', 'Kyber512', 'Kyber768', 'ML-KEM-1024', 'ML-KEM-512', 'ML-KEM-768', 'sntrup761']
Signature algorithms from Kaggle:
['Dilithium2', 'Dilithium3', 'Dilithium5', 'Falcon-1024', 'Falcon-512', 'Falcon-padded-1024', 'Falcon-padded-512', 'MAYO-1', 'MAYO-2', 'MAYO-3', 'MAYO-5', 'ML-DSA-44', 'ML-DSA-65', 'ML-DSA-87', 'OV-III', 'OV-III-pkc', 'OV-III-pkc-skc', 'OV-Ip', 'OV-Ip-pkc', 'OV-Ip-pkc-skc', 'OV-Is', 'OV-Is-pkc', 'OV-Is-pkc-skc', 'OV

In [6]:

# Quick schema sanity & basic cleaning
expected_cols = [
    "algorithm","operation","message_bytes","concurrency",
    "cpu_ghz","aesni","avx2","neon","machine_id","os_name","containerized",
    "latency_ms","throughput_ops","timestamp","notes","trial_id"
]
missing = [c for c in expected_cols if c not in df_local_kem.columns or c not in df_local_sig.columns]
if missing:
    print("Warning: missing expected columns:", missing)

print("Before basic cleaning:", df_local_kem.shape)

# Drop duplicates; filter impossible values
df_local_kem = df_local_kem.drop_duplicates()
df_local_sig = df_local_sig.drop_duplicates()
if "latency_ms" in df_local_kem.columns:
    df_local_kem = df_local_kem[df_local_kem["latency_ms"].notna() & (df_local_kem["latency_ms"] >= 0)]
if "throughput_ops" in df_local_kem.columns:
    df_local_kem = df_local_kem[df_local_kem["throughput_ops"].notna() & (df_local_kem["throughput_ops"] >= 0)]
    
print("After basic cleaning:", df_local_kem.shape)
display(df_local_kem.head(PRINT_ROWS))

print("Before basic cleaning:", df_local_sig.shape)

if "latency_ms" in df_local_sig.columns:
    df_local_sig = df_local_sig[df_local_sig["latency_ms"].notna() & (df_local_sig["latency_ms"] >= 0)]
if "throughput_ops" in df_local_sig.columns:
    df_local_sig = df_local_sig[df_local_sig["throughput_ops"].notna() & (df_local_sig["throughput_ops"] >= 0)]

print("After basic cleaning:", df_local_sig.shape)
display(df_local_sig.head(PRINT_ROWS))


Before basic cleaning: (2340, 18)
After basic cleaning: (2340, 18)


,algorithm,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,crypto_type,correctness,timestamp,notes,trial_id
0,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.7687,1300.83,KEM,True,2025-11-20T22:36:03.573924,NaN,1
1,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.3240,3086.47,KEM,True,2025-11-20T22:36:03.574763,NaN,2
2,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2500,3999.89,KEM,True,2025-11-20T22:36:03.575045,NaN,3
3,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2230,4484.97,KEM,True,2025-11-20T22:36:03.575294,NaN,4
4,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2271,4404.20,KEM,True,2025-11-20T22:36:03.575545,NaN,5


Before basic cleaning: (1800, 18)
After basic cleaning: (1800, 18)


,algorithm,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,crypto_type,correctness,timestamp,notes,trial_id
0,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,1.2456,802.80,Signature,True,2025-11-20T22:32:40.486951,NaN,1
1,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.1651,6058.40,Signature,True,2025-11-20T22:32:40.487678,NaN,2
2,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.1644,6081.43,Signature,True,2025-11-20T22:32:40.487976,NaN,3
3,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.1524,6559.70,Signature,True,2025-11-20T22:32:40.488194,NaN,4
4,Dilithium2,sign,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.1277,7833.12,Signature,True,2025-11-20T22:32:40.488405,NaN,5


In [7]:
import pandas as pd
import numpy as np

def preprocess_kem(kem_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Turn KEM Kaggle data into your unified row-per-operation schema.
    """
    # Columns that should stay as identifiers (everything except *_time_ms)
    kem_id_vars = [
        "algorithm", "ciphertext_length", "shared_secret_length",
        "overhead_bytes", "total_time_ms", "correctness", "type",
        "text_size_kb", "text_length_bytes", "run_id", "timestamp",
        "security_level_bits", "security_level"
    ]
    # Only keep ones that actually exist (defensive)
    kem_id_vars = [c for c in kem_id_vars if c in kem_raw.columns]

    kem_value_vars = ["keygen_time_ms", "encap_time_ms", "decap_time_ms"]
    kem_value_vars = [c for c in kem_value_vars if c in kem_raw.columns]

    # Long format: one row per (algorithm, operation, run_id, message size)
    kem_long = kem_raw.melt(
        id_vars=kem_id_vars,
        value_vars=kem_value_vars,
        var_name="operation",
        value_name="latency_ms"
    )

    # "keygen_time_ms" -> "keygen"
    kem_long["operation"] = kem_long["operation"].str.replace("_time_ms", "", regex=False)

    df_kem = kem_long.copy()

    # Map to your schema
    #df_kem["alg_family"] = df_kem["algorithm"].astype(str).str.extract(r"([A-Za-z\-\+]+)", expand=False)
    #df_kem["param_set"]  = df_kem["algorithm"].astype(str).str.extract(r"(\d+)", expand=False)

    # Message size
    df_kem["message_bytes"] = df_kem["text_length_bytes"].fillna(0)

    # Latency and throughput
    df_kem["latency_ms"] = pd.to_numeric(df_kem["latency_ms"], errors="coerce")
    df_kem["throughput_ops"] = 1000.0 / df_kem["latency_ms"].clip(lower=1e-6)

    # Fields Kaggle does not have → fill with defaults
    df_kem["concurrency"] = 1
    df_kem["cpu_ghz"] = 0.0
    df_kem["aesni"] = 0
    df_kem["avx2"] = 0
    df_kem["neon"] = 0
    df_kem["machine_id"] = "kaggle_kem"
    df_kem["os_name"] = "unknown"
    df_kem["containerized"] = 0
    df_kem["notes"] = "from_kaggle_kem"
    df_kem["trial_id"] = df_kem["run_id"]
    df_kem["crypto_type"] = df_kem["type"]

    cols = [
        "operation", "message_bytes", "concurrency",
        "cpu_ghz", "aesni", "avx2", "neon", "machine_id", "os_name", "containerized",
        "latency_ms", "throughput_ops", "timestamp", "notes", "trial_id",
        # extras to keep if you want them in the DF
        "algorithm", "crypto_type", "security_level_bits", "security_level",
        "ciphertext_length", "shared_secret_length", "overhead_bytes",
        "total_time_ms", "correctness"
    ]
    cols = [c for c in cols if c in df_kem.columns]

    return df_kem[cols]


def preprocess_sig(sig_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Turn Signature Kaggle data into your unified row-per-operation schema.
    """
    sig_id_vars = [
        "algorithm", "text_size_kb",
        "signature_length", "overhead_bytes", "total_time_ms",
        "correctness", "type", "text_length_bytes", "run_id",
        "timestamp", "security_level_bits", "security_level"
    ]
    sig_id_vars = [c for c in sig_id_vars if c in sig_raw.columns]

    sig_value_vars = ["keygen_time_ms", "sign_time_ms", "verify_time_ms"]
    sig_value_vars = [c for c in sig_value_vars if c in sig_raw.columns]

    sig_long = sig_raw.melt(
        id_vars=sig_id_vars,
        value_vars=sig_value_vars,
        var_name="operation",
        value_name="latency_ms"
    )

    sig_long["operation"] = sig_long["operation"].str.replace("_time_ms", "", regex=False)

    df_sig = sig_long.copy()

    #df_sig["alg_family"] = df_sig["algorithm"].astype(str).str.extract(r"([A-Za-z\-\+]+)", expand=False)
    #df_sig["param_set"]  = df_sig["algorithm"].astype(str).str.extract(r"(\d+)", expand=False)

    df_sig["message_bytes"] = df_sig["text_length_bytes"].fillna(0)

    df_sig["latency_ms"] = pd.to_numeric(df_sig["latency_ms"], errors="coerce")
    df_sig["throughput_ops"] = 1000.0 / df_sig["latency_ms"].clip(lower=1e-6)

    df_sig["concurrency"] = 1
    df_sig["cpu_ghz"] = 0.0
    df_sig["aesni"] = 0
    df_sig["avx2"] = 0
    df_sig["neon"] = 0
    df_sig["machine_id"] = "kaggle_sig"
    df_sig["os_name"] = "unknown"
    df_sig["containerized"] = 0
    df_sig["notes"] = "from_kaggle_signature"
    df_sig["trial_id"] = df_sig["run_id"]
    df_sig["crypto_type"] = df_sig["type"]

    cols = [
        "operation", "message_bytes", "concurrency",
        "cpu_ghz", "aesni", "avx2", "neon", "machine_id", "os_name", "containerized",
        "latency_ms", "throughput_ops", "timestamp", "notes", "trial_id",
        "algorithm", "crypto_type", "security_level_bits", "security_level",
        "signature_length", "overhead_bytes", "total_time_ms", "correctness"
    ]
    cols = [c for c in cols if c in df_sig.columns]

    return df_sig[cols]


In [8]:
df_kem_ready = preprocess_kem(kem_raw)
df_sig_ready = preprocess_sig(sig_raw)

print("Before Preprocess:", df_kem_ready.shape, df_sig_ready.shape)
df_kaggle_all = pd.concat([df_kem_ready, df_sig_ready], ignore_index=True)
print("Combined Kaggle rows:", len(df_kaggle_all))
df_kaggle_all.head()

# Build lookup: one row per algorithm with its security levels from Kaggle
sec_lookup = (
    df_kaggle_all[["algorithm", "security_level_bits", "security_level"]]
    .dropna(subset=["security_level_bits", "security_level"])
    .drop_duplicates(subset=["algorithm"])
)
print("Security lookup size:", sec_lookup.shape)
display(sec_lookup.head())

print("After Preprocess:", df_kaggle_all.shape)
display(df_kaggle_all.head(PRINT_ROWS))
print(list(df_kaggle_all.columns))

# Attach security levels to local KEM rows
df_local_kem = df_local_kem.merge(
    sec_lookup,
    on="algorithm",
    how="left",
)

# Attach security levels to local Signature rows
df_local_sig = df_local_sig.merge(
    sec_lookup,
    on="algorithm",
    how="left",
)

df_all = pd.concat([df_local_kem, df_local_sig, df_kaggle_all], ignore_index=True)
print("Unified dataset shape:", df_all.shape)
display(df_all.head())

Before Preprocess: (111012, 24) (214368, 23)
Combined Kaggle rows: 325380
Security lookup size: (85, 3)


,algorithm,security_level_bits,security_level
0,BIKE-L1,128,1
1,BIKE-L3,192,2
2,BIKE-L5,256,3
3,Classic-McEliece-348864,128,1
4,Classic-McEliece-348864f,128,1


After Preprocess: (325380, 25)


,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,timestamp,notes,trial_id,algorithm,crypto_type,security_level_bits,security_level,ciphertext_length,shared_secret_length,overhead_bytes,total_time_ms,correctness,signature_length
0,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,8.220911,121.641020,2025-04-07T09:04:19.790636,from_kaggle_kem,1,BIKE-L1,KEM,128,1,1573.0,32.0,1541,14.823914,True,NaN
1,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,21.829844,45.808849,2025-04-07T09:04:19.829883,from_kaggle_kem,1,BIKE-L3,KEM,192,2,3115.0,32.0,3083,39.199114,True,NaN
2,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,46.767712,21.382273,2025-04-07T09:04:19.916316,from_kaggle_kem,1,BIKE-L5,KEM,256,3,5154.0,32.0,5122,86.393833,True,NaN
3,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,93.240976,10.724898,2025-04-07T09:04:20.027250,from_kaggle_kem,1,Classic-McEliece-348864,KEM,128,1,96.0,32.0,64,110.874891,True,NaN
4,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,29.181004,34.268869,2025-04-07T09:04:20.072888,from_kaggle_kem,1,Classic-McEliece-348864f,KEM,128,1,96.0,32.0,64,45.579910,True,NaN


['operation', 'message_bytes', 'concurrency', 'cpu_ghz', 'aesni', 'avx2', 'neon', 'machine_id', 'os_name', 'containerized', 'latency_ms', 'throughput_ops', 'timestamp', 'notes', 'trial_id', 'algorithm', 'crypto_type', 'security_level_bits', 'security_level', 'ciphertext_length', 'shared_secret_length', 'overhead_bytes', 'total_time_ms', 'correctness', 'signature_length']
Unified dataset shape: (329520, 25)


,algorithm,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,crypto_type,correctness,timestamp,notes,trial_id,security_level_bits,security_level,ciphertext_length,shared_secret_length,overhead_bytes,total_time_ms,signature_length
0,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.7687,1300.83,KEM,True,2025-11-20T22:36:03.573924,NaN,1,128,1,NaN,NaN,NaN,NaN,NaN
1,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.3240,3086.47,KEM,True,2025-11-20T22:36:03.574763,NaN,2,128,1,NaN,NaN,NaN,NaN,NaN
2,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2500,3999.89,KEM,True,2025-11-20T22:36:03.575045,NaN,3,128,1,NaN,NaN,NaN,NaN,NaN
3,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2230,4484.97,KEM,True,2025-11-20T22:36:03.575294,NaN,4,128,1,NaN,NaN,NaN,NaN,NaN
4,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2271,4404.20,KEM,True,2025-11-20T22:36:03.575545,NaN,5,128,1,NaN,NaN,NaN,NaN,NaN


In [9]:

# Feature engineering
import numpy as np

# Composite algorithm identifier
if set(["alg_family","param_set","operation"]).issubset(df_all.columns):
    df_all["alg_param"] = df_all["alg_family"].astype(str) + "_" + df_all["param_set"].astype(str) + "_" + df_all["operation"].astype(str)

# Ensure types
for col in ["message_bytes","concurrency","cpu_ghz","aesni","avx2","neon","containerized"]:
    if col in df_all.columns:
        df_all[col] = pd.to_numeric(df_all[col], errors="coerce")

# Log-transform latency for modeling stability (kept as an extra feature)
if "latency_ms" in df_all.columns:
    df_all["log_latency"] = np.log1p(df_all["latency_ms"])

# Keep a clean feature list (drop targets, ids, free-text)
drop_cols = {TARGET, "log_latency", "timestamp", "notes", "trial_id"}
features = [c for c in df_all.columns if c not in drop_cols]
print("Candidate features:", features)
# Local harness
display(df_all[df_all["machine_id"] == "MacBook-Pro-4.local"].head(5))

# Kaggle KEM
display(df_all[df_all["machine_id"] == "kaggle_kem"].head(5))

# Kaggle Signature
display(df_all[df_all["machine_id"] == "kaggle_sig"].head(5))


Candidate features: ['algorithm', 'operation', 'message_bytes', 'concurrency', 'cpu_ghz', 'aesni', 'avx2', 'neon', 'machine_id', 'os_name', 'containerized', 'throughput_ops', 'crypto_type', 'correctness', 'security_level_bits', 'security_level', 'ciphertext_length', 'shared_secret_length', 'overhead_bytes', 'total_time_ms', 'signature_length']


,algorithm,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,crypto_type,correctness,timestamp,notes,trial_id,security_level_bits,security_level,ciphertext_length,shared_secret_length,overhead_bytes,total_time_ms,signature_length,log_latency
0,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.7687,1300.83,KEM,True,2025-11-20T22:36:03.573924,NaN,1,128,1,NaN,NaN,NaN,NaN,NaN,0.570245
1,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.3240,3086.47,KEM,True,2025-11-20T22:36:03.574763,NaN,2,128,1,NaN,NaN,NaN,NaN,NaN,0.280657
2,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2500,3999.89,KEM,True,2025-11-20T22:36:03.575045,NaN,3,128,1,NaN,NaN,NaN,NaN,NaN,0.223144
3,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2230,4484.97,KEM,True,2025-11-20T22:36:03.575294,NaN,4,128,1,NaN,NaN,NaN,NaN,NaN,0.201307
4,BIKE-L1,keygen,0,1,2.0,1,1,0,MacBook-Pro-4.local,darwin,0,0.2271,4404.20,KEM,True,2025-11-20T22:36:03.575545,NaN,5,128,1,NaN,NaN,NaN,NaN,NaN,0.204654


,algorithm,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,crypto_type,correctness,timestamp,notes,trial_id,security_level_bits,security_level,ciphertext_length,shared_secret_length,overhead_bytes,total_time_ms,signature_length,log_latency
4140,BIKE-L1,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,8.220911,121.641020,KEM,True,2025-04-07T09:04:19.790636,from_kaggle_kem,1,128,1,1573.0,32.0,1541.0,14.823914,NaN,2.221474
4141,BIKE-L3,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,21.829844,45.808849,KEM,True,2025-04-07T09:04:19.829883,from_kaggle_kem,1,192,2,3115.0,32.0,3083.0,39.199114,NaN,3.128069
4142,BIKE-L5,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,46.767712,21.382273,KEM,True,2025-04-07T09:04:19.916316,from_kaggle_kem,1,256,3,5154.0,32.0,5122.0,86.393833,NaN,3.866350
4143,Classic-McEliece-348864,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,93.240976,10.724898,KEM,True,2025-04-07T09:04:20.027250,from_kaggle_kem,1,128,1,96.0,32.0,64.0,110.874891,NaN,4.545855
4144,Classic-McEliece-348864f,keygen,512,1,0.0,0,0,0,kaggle_kem,unknown,0,29.181004,34.268869,KEM,True,2025-04-07T09:04:20.072888,from_kaggle_kem,1,128,1,96.0,32.0,64.0,45.579910,NaN,3.407213


,algorithm,operation,message_bytes,concurrency,cpu_ghz,aesni,avx2,neon,machine_id,os_name,containerized,latency_ms,throughput_ops,crypto_type,correctness,timestamp,notes,trial_id,security_level_bits,security_level,ciphertext_length,shared_secret_length,overhead_bytes,total_time_ms,signature_length,log_latency
115152,Dilithium2,keygen,512,1,0.0,0,0,0,kaggle_sig,unknown,0,0.037193,26886.564103,Signature,True,2025-04-07T09:04:23.004269,from_kaggle_signature,1,128,1,NaN,NaN,1908.0,0.339031,2420.0,0.036518
115153,Dilithium3,keygen,512,1,0.0,0,0,0,kaggle_sig,unknown,0,0.214815,4655.165372,Signature,True,2025-04-07T09:04:23.004649,from_kaggle_signature,1,192,2,NaN,NaN,2781.0,0.360966,3293.0,0.194592
115154,Dilithium5,keygen,512,1,0.0,0,0,0,kaggle_sig,unknown,0,0.073910,13530.012903,Signature,True,2025-04-07T09:04:23.004884,from_kaggle_signature,1,256,3,NaN,NaN,4083.0,0.204802,4595.0,0.071306
115155,ML-DSA-44,keygen,512,1,0.0,0,0,0,kaggle_sig,unknown,0,0.069141,14463.117241,Signature,True,2025-04-07T09:04:23.005105,from_kaggle_signature,1,128,1,NaN,NaN,1908.0,0.207186,2420.0,0.066856
115156,ML-DSA-65,keygen,512,1,0.0,0,0,0,kaggle_sig,unknown,0,0.093937,10645.441624,Signature,True,2025-04-07T09:04:23.005454,from_kaggle_signature,1,192,2,NaN,NaN,2797.0,0.340939,3309.0,0.089783


In [10]:

# Train/test split
from sklearn.model_selection import train_test_split

y = df_all[TARGET].copy()
X = df_all[features].copy()

# Example group-wise split: hold out one machine to test generalization across hardware
group_col = "machine_id" if "machine_id" in X.columns else None
if group_col and X[group_col].nunique() > 1:
    held_out = X[group_col].unique()[-1]
    train_idx = X[group_col] != held_out
    test_idx  = X[group_col] == held_out
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    print(f"Held-out machine: {held_out} | train={X_train.shape} test={X_test.shape}")
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("Random split:", X_train.shape, X_test.shape)

#mixed_type_cols = []
#for c in cat_cols:
    #types_in_col = X_train[c].map(type).unique()
    #if len(types_in_col) > 1:
        #print(c, "has types:", types_in_col)
        #mixed_type_cols.append(c)


Held-out machine: kaggle_sig | train=(115152, 21) test=(214368, 21)


## 2) Modeling
We train a baseline linear model, a flexible tree-based model, and quantile regressors for p95/p99.

In [11]:

# 2.1 Baseline: Regularized Linear (Ridge)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

pre = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
])

ridge = Pipeline([("pre", pre), ("model", Ridge(alpha=1.0, random_state=42))])
ridge.fit(X_train, y_train)
pred_ridge = ridge.predict(X_test)
mae = mean_absolute_error(y_test, pred_ridge)
mse = mean_squared_error(y_test, pred_ridge)
rmse = np.sqrt(mse) # Use numpy.sqrt to get the square root
print(f"Ridge → MAE={mae:.4f} | RMSE={rmse:.4f}")


/Users/newspeed/.venvs/ds/lib/python3.12/site-packages/sklearn/utils/extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/newspeed/.venvs/ds/lib/python3.12/site-packages/sklearn/utils/extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/newspeed/.venvs/ds/lib/python3.12/site-packages/sklearn/utils/extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


ValueError: Input X contains NaN.
Ridge does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:

# 2.2 Flexible model: RandomForest (uses minimal preprocessing)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]
pre2 = ColumnTransformer([("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)], remainder="passthrough")

rf = Pipeline([
    ("pre", pre2),
    ("rf", RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1))
])
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, pred_rf)
mse_rf = mean_squared_error(y_test, pred_rf)
rmse_rf = np.sqrt(mse_rf) # Use numpy.sqrt to get the square root
print(f"RandomForest → MAE={mae_rf:.4f} | RMSE={rmse_rf:.4f}")


In [ ]:

# 2.3 Optional: XGBoost if installed
try:
    import xgboost as xgb
    use_xgb = True
except Exception:
    use_xgb = False
    print("xgboost not installed; skipping XGB model.")

if use_xgb:
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.metrics import mean_absolute_error, mean_squared_error

    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X_train.columns if c not in num_cols]
    pre3 = ColumnTransformer([("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)], remainder="passthrough")

    xgb_model = xgb.XGBRegressor(
        n_estimators=600, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1
    )
    xgb_pipe = Pipeline([("pre", pre3), ("xgb", xgb_model)])
    xgb_pipe.fit(X_train, y_train)
    pred_xgb = xgb_pipe.predict(X_test)
    mae_xgb = mean_absolute_error(y_test, pred_xgb)
    mse_xgb = mean_squared_error(y_test, pred_xgb)
    rmse_xgb = np.sqrt(rmse_xgb) # Use numpy.sqrt to get the square root
    print(f"XGBoost → MAE={mae_xgb:.4f} | RMSE={rmse_xgb:.4f}")


In [ ]:

# 2.4 Quantile regression (p50/p95/p99) with GradientBoostingRegressor
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor

quantiles = (0.5, 0.95, 0.99)
qr_models = {}
X_train_dm = pd.get_dummies(X_train)
X_test_dm  = pd.get_dummies(X_test).reindex(columns=X_train_dm.columns, fill_value=0)
for q in quantiles:
    m = GradientBoostingRegressor(loss="quantile", alpha=q, random_state=42)
    m.fit(X_train_dm, y_train)
    qr_models[q] = m

q = 0.95
q_pred = qr_models[q].predict(X_test_dm)
emp_cov = float((y_test <= q_pred).mean())
print(f"Quantile {q:.2f} empirical coverage: {emp_cov:.3f}")


## 3) Visualization
Simple plots to understand distributions, compare algorithms, and inspect model behavior.

In [ ]:

import matplotlib.pyplot as plt

# 3.1 Latency/Throughput histogram
plt.figure()
df[TARGET].hist(bins=50)
plt.title(f"{TARGET} distribution")
plt.xlabel(TARGET); plt.ylabel("count")
plt.show()


In [ ]:

# 3.2 Boxplot by alg/param/operation (using 'alg_param' if present)
if "alg_param" in df.columns and TARGET in df.columns:
    plt.figure()
    groups = df.groupby("alg_param")[TARGET]
    data = [g.values for _, g in groups]
    labels = [k for k, _ in groups]
    plt.boxplot(data, tick_labels=labels, vert=True, showfliers=False)
    plt.title(f"{TARGET} by alg/param/operation")
    plt.xticks(rotation=90)
    plt.ylabel(TARGET)
    plt.tight_layout()
    plt.show()


In [ ]:

# 3.3 Scatter: concurrency vs target
if "concurrency" in df.columns and TARGET in df.columns:
    plt.figure()
    plt.scatter(df["concurrency"], df[TARGET], s=8, alpha=0.5)
    plt.title(f"Concurrency vs {TARGET}")
    plt.xlabel("concurrency"); plt.ylabel(TARGET)
    plt.show()


In [ ]:

# 3.4 Feature importance from RandomForest
try:
    pre = rf.named_steps["pre"]
    cat_features = pre.transformers_[0][2]
    from sklearn.preprocessing import OneHotEncoder
    import numpy as np

    ohe = pre.transformers_[0][1]
    ohe_cols = ohe.get_feature_names_out(cat_features)
    num_cols = [c for c in X_train.columns if c not in cat_features]
    colnames = list(ohe_cols) + num_cols

    importances = rf.named_steps["rf"].feature_importances_
    order = np.argsort(importances)[::-1][:15]

    plt.figure()
    plt.bar(range(len(order)), importances[order])
    plt.xticks(range(len(order)), [colnames[i] for i in order], rotation=90)
    plt.title("RandomForest top feature importances")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Feature importance plot skipped:", e)


In [ ]:

# 3.5 What-if helper using your best model (rf/xgb/ridge)
best_model = rf  # change if another model performs better

def predict_target(example: dict, model, columns):
    import pandas as pd
    X_ex = pd.DataFrame([example], columns=columns)
    return float(model.predict(X_ex)[0])

example = {
    "alg_family": "Dilithium3",
    "param_set": "",
    "operation": "sign",
    "message_bytes": 1024,
    "concurrency": 64,
    "cpu_ghz": 0.0,
    "aesni": 0, "avx2": 0, "neon": 0,
    "machine_id": X_train["machine_id"].iloc[0] if "machine_id" in X_train.columns else "",
    "os_name": X_train["os_name"].iloc[0] if "os_name" in X_train.columns else "",
    "containerized": 0,
    "alg_param": "Dilithium3__sign"
}
try:
    print("What-if predicted", TARGET, "=", predict_target(example, best_model, X_train.columns))
except Exception as e:
    print("What-if example failed (adjust keys to match your columns):", e)


---
**Tip:** To predict throughput instead, set `TARGET = 'throughput_ops'` above and rerun all.